In [13]:
import os

from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI 

In [15]:
llm = ChatGoogleGenerativeAI(
    api_key = api_key,
    model = "gemini-2.0-flash"
)

In [ ]:
import os
import re
import sqlite3
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
import dateparser

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.0-flash"
)

appointment_pattern = {
    "user_name": r"(?:name[:\s]+)([A-Za-z\s]+)|(?:my name is\s+)([A-Za-z\s]+)|(?:i am\s+)([A-Za-z\s]+)",
    "date": r"(?:date[:\s]+)([A-Za-z0-9\s,]+)|(?:on\s+)([A-Za-z0-9\s,]+)",
    "time": r"(?:time[:\s]+)([0-9:]+\s*(?:AM|PM|am|pm)?)|(?:at\s+)([0-9:]+\s*(?:AM|PM|am|pm)?)",
    "email": r"(?:email[:\s]+)([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)|([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",
    "appointment_type": r"(?:appointment_type[:\s]+)(Data Science|AI\/ML|Application Development|Database Development)|(?:for\s+)(Data Science|AI\/ML|Application Development|Database Development)|(?:regarding\s+)(Data Science|AI\/ML|Application Development|Database Development)"
}

system_prompt = """
        You are an appointment booking assistant. Your task is to:
        1. Help users book appointments by collecting information step-by-step.
        2. Begin by greeting the user and asking for their name.
        3. Once the name is provided, ask for their email address (this is required).
        4. After obtaining the email, ask for the appointment date.
        5. Then, ask for the appointment time.
        6. Finally, ask for the appointment_reason.
        7. Additionally, assist users in retrieving their appointment information if needed.
        8. Respond in a friendly and helpful way.

        Important: Ensure you always ask for an email address if it's not provided, as it is required for all appointments.

        - Only use information explicitly provided by the user. Do not add or assume any details that have not been given.
        - If any required information (such as name, email, date, time, or appointment_reason) is missing or ambiguous, clearly state which details are needed and ask clarifying questions.
        - Verify the validity of provided details before generating appointment records.
        - When retrieving appointment information, only reference data that has been confirmed by the user.
        - Refrain from generating extraneous or fictional details.

        When you identify appointment details, format your response with JSON-like tags:
         
        <APPOINTMENT_DETAILS>
        name: [extracted name]
        email: [extracted email]
        date: [extracted date in DD-MM-YYYY format]
        time: [extracted time in HH:MM format]
        appointment_reason: [extracted appointment_reason]
        action: [book/retrieve]
        </APPOINTMENT_DETAILS>
        
        If any required information is missing, clearly state which details are needed and do not use the JSON-like format."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])
def init_db():
    conn = sqlite3.connect("booking.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS booking (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_name TEXT NOT NULL,
            date TEXT NOT NULL,
            time TEXT NOT NULL,
            email TEXT NOT NULL,
            appointment_type TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

def parse_dates(text):
    dt = dateparser.parse(text)
    if dt:
        return [dt]
    return []

def extract_appointment_details(user_input):
    appointment_details = {}
    
    for field, pattern in appointment_pattern.items():
        match = re.search(pattern, user_input, re.IGNORECASE)
        if match:
            caught_group = next((g for g in match.groups() if g is not None), None)
            if caught_group:
                appointment_details[field] = caught_group.strip()
    
    if "date" not in appointment_details:
        try:
            dates = parse_dates(user_input)
            if dates:
                appointment_details["date"] = dates[0].strftime("%Y-%m-%d")
        except Exception as e:
            print(f"Date parsing error: {e}")
    
    return appointment_details

def format_time(time_str):
    try:
        time_str = time_str.strip().lower()
        
        formats = [
            "%I:%M %p",
            "%I:%M%p",
            "%H:%M",
            "%I %p"      
        ]
        
        for fmt in formats:
            try:
                time_obj = datetime.strptime(time_str, fmt)
                return time_obj.strftime("%H:%M")
            except ValueError:
                continue
        
        if ":" not in time_str:
            match = re.match(r"(\d+)\s*(am|pm)", time_str)
            if match:
                hour, meridiem = match.groups()
                hour = int(hour)
                if meridiem == "pm" and hour < 12:
                    hour += 12
                elif meridiem == "am" and hour == 12:
                    hour = 0
                return f"{hour:02d}:00"
        
        return time_str
    except Exception as e:
        print(f"Error formatting time: {e}")
        return time_str

def save_appointment(details, message_history):
    if not details:
        return "No appointment details found to save."
    
    if 'time' in details:
        try:
            details['time'] = format_time(details['time'])
        except Exception as e:
            print(f"Error formatting time: {e}")
    
    try:
        conn = sqlite3.connect("booking.db")
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO booking (user_name, date, time, email, appointment_type)
            VALUES (?, ?, ?, ?, ?)
        ''', (details['user_name'], details['date'], details['time'], details['email'], details['appointment_type']))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Database error: {e}")
        return "Failed to save appointment due to a database error."
    
    appointment_str = "APPOINTMENT_DETAILS: " + ", ".join([f"{k}: {v}" for k, v in details.items()])
    message_history.add_user_message("Save my appointment")
    message_history.add_ai_message(appointment_str)
    
    return details

def retrieve_appointments(query=None):
    conn = sqlite3.connect("booking.db")
    cursor = conn.cursor()

    where_clauses = []
    parameters = []
    is_next = False
    email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
    email_match = re.search(email_pattern, query) if query else None
    if email_match:
        email_filter = email_match.group(0)
        where_clauses.append("email LIKE ?")
        parameters.append('%' + email_filter + '%')

    query_lower = query.lower() if query else ""
    if "tomorrow" in query_lower:
        tomorrow = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
        where_clauses.append("date = ?")
        parameters.append(tomorrow)
    elif "today" in query_lower:
        today = datetime.now().strftime("%Y-%m-%d")
        where_clauses.append("date = ?")
        parameters.append(today)
    elif "next appointment" in query_lower or "next" in query_lower:
        today = datetime.now().strftime("%Y-%m-%d")
        where_clauses.append("date >= ?")
        parameters.append(today)
        is_next = True

    query_str = "SELECT user_name, date, time, email, appointment_type FROM appointments"
    if where_clauses:
        query_str += " WHERE " + " AND ".join(where_clauses)

    if is_next:
        query_str += " ORDER BY date ASC LIMIT 1"

    cursor.execute(query_str, parameters)
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        return "No appointments found."
    
    appointments_list = []
    for row in rows:
        appointment = f"Name: {row[0]}, Date: {row[1]}, Time: {row[2]}, Email: {row[3]}, Service: {row[4]}"
        appointments_list.append(appointment)
    
    if is_next:
        return "Your next appointment is:\n" + "\n".join(appointments_list)
    
    if query and email_match:
        return "Found these appointments matching your email:\n" + "\n".join(appointments_list)
    
    return "Here are your appointments:\n" + "\n".join(appointments_list)

def has_all_required_details(details):
    required_fields = ["user_name", "date", "time", "email", "appointment_type"]
    return all(field in details for field in required_fields)

def is_booking_request(user_input):
    booking_keywords = ["book", "schedule", "make", "set", "create", "arrange", "appointment"]
    return any(keyword in user_input.lower() for keyword in booking_keywords)

def is_retrieval_request(user_input):
    retrieval_keywords = ["show", "get", "find", "view", "check", "see", "tell", "when is", "do i have"]
    appointment_keywords = ["appointment", "schedule", "booking"]
    
    has_retrieval = any(keyword in user_input.lower() for keyword in retrieval_keywords)
    has_appointment = any(keyword in user_input.lower() for keyword in appointment_keywords)
    
    return has_retrieval and has_appointment

def get_next_question(details):
    if "user_name" not in details:
        return "Could you please tell me your name?"
    elif "date" not in details:
        return "On what date would you like to schedule your appointment?"
    elif "time" not in details:
        return "What time would work best for you?"
    elif "email" not in details:
        return "Please provide your email address for the appointment confirmation."
    elif "appointment_type" not in details:
        return "What type of service are you interested in? Sabir specializes in: Data Science, AI/ML, Application Development, or Database Development."
    else:
        return None

conversation_states = {}

def process_user_input(user_input, session_id="default"):
    if session_id not in conversation_states:
        conversation_states[session_id] = {
            "message_history": ChatMessageHistory(),
            "current_details": {},
            "collecting_info": False
        }
    
    state = conversation_states[session_id]
    message_history = state["message_history"]
    current_details = state["current_details"]
    
    new_details = extract_appointment_details(user_input)
    current_details.update(new_details)
    
    if is_booking_request(user_input) or state["collecting_info"]:
        state["collecting_info"] = True
        
        if has_all_required_details(current_details):
            save_result = save_appointment(current_details, message_history)
            if isinstance(save_result, str):
                return save_result
            
            confirmation = f"""
Great! I've booked your appointment.

Appointment Details:
- Name: {current_details['user_name']}
- Date: {current_details['date']}
- Time: {current_details['time']}
- Email: {current_details['email']}
- Service: {current_details['appointment_type']}

Thank you for scheduling with us. You will receive a confirmation email shortly.
Is there anything else I can help you with?
"""
            state["collecting_info"] = False
            state["current_details"] = {}
            
            message_history.add_user_message(user_input)
            message_history.add_ai_message(confirmation)
            
            return confirmation
        else:
            next_question = get_next_question(current_details)
            message_history.add_user_message(user_input)
            message_history.add_ai_message(next_question)
            
            return next_question
    
    elif is_retrieval_request(user_input):
        response = retrieve_appointments(user_input)
        message_history.add_user_message(user_input)
        message_history.add_ai_message(response)
        return response
    
    else:
        chain = prompt | llm | StrOutputParser()
        chain_with_history = RunnableWithMessageHistory(
            chain,
            lambda session_id: message_history,
            input_messages_key="input",
            history_messages_key="history",
        )
        response = chain_with_history.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        return response

if __name__ == "__main__":
    init_db() 
    print("Welcome to the Appointment Booking System with Sabir!")
    print("You can book appointments for Data Science, AI/ML, Application Development, or Database Development.")
    print("Type 'exit' to quit.\n")
    
    session_id = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Thank you for using the Appointment Booking System. Goodbye!")
            break
        
        response = process_user_input(user_input, session_id)
        print(f"Bot: {response}")


Welcome to the Appointment Booking System with Sabir!
You can book appointments for Data Science, AI/ML, Application Development, or Database Development.
Type 'exit' to quit.

Bot: Hi there! I can help you book an appointment with Sabir. First, can I get your name, please?
Bot: Okay Fahim, I need a bit more information to book the appointment.

What date would you like to book your appointment?
Bot: Thanks! And what time would you like to book your appointment for on October 2, 2025?
Bot: I need a time for your appointment on October 2, 2025. Please provide the time you would like to book.
Bot: Great. And can I get your email address?
Bot: Okay, Fahim. Last question: What type of appointment would you like to book with Sabir? 

The options are: Data Science, AI/ML, Application Development, or Database Development.
Bot: Okay Fahim, just to confirm, you would like to book an appointment with Sabir for Data Science on October 2, 2025, at 10:00 AM, and your email address is fahim@gmail.co